In [52]:
import os
import json
from collections import defaultdict
import random
from koala.utils.utils import load_jsonl, save_jsonl, save_tsv, mkdir, load_tsv
random.seed(42)

In [15]:
processed_data_dir = f"data/processed/training/"
grouped_data_file_name = "grouped_data.jsonl"
ranked_grouped_data_file_name = "grouped_data.ranked.jsonl"
ranked_grouped_train_data_file_name = "grouped_data.ranked.train.jsonl"
ranked_grouped_valid_data_file_name = "grouped_data.ranked.valid.jsonl"
collection_file_name = "collection.tsv"
language = "cn"
style_name = "强制性"
style_definition = "语气越强硬、措施或要求越详细具体、涉及到硬性的数字标准，则文本的“强制性”越强。"
raw_data_path = "data/raw/raw_data.json"

In [ ]:
# todo

# few shot or not

# 保存raw_data

In [75]:
data_dir = "C:/Users/Alexandra/Desktop/working_env/ranker/myMethod-main/data/data_to_be_ranked/"

In [76]:
os.listdir(data_dir)

['农业_中央_batch-paras.txt',
 '农业_市级_batch-paras.txt',
 '农业_省级_batch-paras.txt',
 '医疗_中央_batch-paras.txt',
 '医疗_市级_batch-paras.txt',
 '医疗_省级_batch-paras.txt',
 '就业_中央_batch-paras.txt',
 '就业_市级_batch-paras.txt',
 '就业_省级_batch-paras.txt',
 '政府_中央_batch-paras.txt',
 '政府_市级_batch-paras.txt',
 '政府_省级_batch-paras.txt',
 '教育_中央_batch-paras.txt',
 '教育_市级_batch-paras.txt',
 '教育_省级_batch-paras.txt',
 '文化_中央_batch-paras.txt',
 '文化_市级_batch-paras.txt',
 '文化_省级_batch-paras.txt',
 '民生_中央_batch-paras.txt',
 '民生_市级_batch-paras.txt',
 '民生_省级_batch-paras.txt',
 '生态_中央_batch-paras.txt',
 '生态_市级_batch-paras.txt',
 '生态_省级_batch-paras.txt',
 '科技_中央_batch-paras.txt',
 '科技_市级_batch-paras.txt',
 '科技_省级_batch-paras.txt',
 '经济_中央_batch-paras.txt',
 '经济_市级_batch-paras.txt',
 '经济_省级_batch-paras.txt']

In [77]:
classification_idx = {
    "农业": "agri",
    "经济": "econ",
    "医疗": "med",
    "就业": "emp",
    "教育": "edu",
    "政府": "gov",
    "文化": "cul",
    "民生": "liv",
    "科技": "tech",
    "生态": "eco"
}

In [78]:
category2texts = defaultdict(list)

for file in os.listdir(data_dir):
    data_path = data_dir + file
    print(data_path)
    category = file.split("_")[0]
    category_name = classification_idx[category]
    with open(data_path, "r", encoding="utf-8") as fp:
        while True:
            line = fp.readline()
            if not line:
                break
            info = json.loads(line)
            texts = info["batch_paras"]
            category2texts[category_name].extend(texts)

for k, v in category2texts.items():
    print(k, len(v))
    # print(len(v))
    # category2texts[category].append(texts)

C:/Users/Alexandra/Desktop/working_env/ranker/myMethod-main/data/data_to_be_ranked/农业_中央_batch-paras.txt
C:/Users/Alexandra/Desktop/working_env/ranker/myMethod-main/data/data_to_be_ranked/农业_市级_batch-paras.txt
C:/Users/Alexandra/Desktop/working_env/ranker/myMethod-main/data/data_to_be_ranked/农业_省级_batch-paras.txt
C:/Users/Alexandra/Desktop/working_env/ranker/myMethod-main/data/data_to_be_ranked/医疗_中央_batch-paras.txt
C:/Users/Alexandra/Desktop/working_env/ranker/myMethod-main/data/data_to_be_ranked/医疗_市级_batch-paras.txt
C:/Users/Alexandra/Desktop/working_env/ranker/myMethod-main/data/data_to_be_ranked/医疗_省级_batch-paras.txt
C:/Users/Alexandra/Desktop/working_env/ranker/myMethod-main/data/data_to_be_ranked/就业_中央_batch-paras.txt
C:/Users/Alexandra/Desktop/working_env/ranker/myMethod-main/data/data_to_be_ranked/就业_市级_batch-paras.txt
C:/Users/Alexandra/Desktop/working_env/ranker/myMethod-main/data/data_to_be_ranked/就业_省级_batch-paras.txt
C:/Users/Alexandra/Desktop/working_env/ranker/myMethod-

In [79]:
# sample it
category2texts_sampled = {}

for category, texts in category2texts.items():
    texts = set(texts)
    sample_texts = random.sample(texts, 800)
    category2texts_sampled[category] = sample_texts

In [80]:
line_list = []
for category, texts in category2texts_sampled.items():
    for text in texts:
        text = text.replace("\t", " ")
        line = {"domain": category, "text": text}
        line_list.append(line)

In [81]:
save_jsonl(line_list, raw_data_path)

>>> save jsonl to data/raw/raw_data.jsonl: 8000


In [82]:
# 教育，文化，科技，民生，生态

In [83]:
classified_data_path = "C:/Users/Alexandra/Desktop/working_env/ranker/myMethod-main/data/classification_data/province_key_paras_classified.txt"

In [84]:
with open(classified_data_path, "r", encoding="utf-8") as fp:
    lines = fp.readlines()
    for line in lines:
        info = json.loads(line)
        key_paras = info["key_paras"]
        for category, paras in key_paras.items():
            print(category)
        break

民生
其他
科技
生态


# Get the training data (raw_data -> processed_data)

In [85]:
# get the data to be ranked
raw_data = load_jsonl(raw_data_path)

In [86]:
raw_data[0]

{'domain': 'agri',
 'text': '农业是安天下、稳民心的战略产业。解决好“三农”问题，事关经济社会发展全局。坚持城乡统筹，以工促农、以城带乡，突出农民持续增收和提高农业综合生产能力、确保粮食安全两大主题，加快实现引黄灌区由传统农业向现代农业、南部山区由传统种植业向现代生态畜牧业和种植业相结合转变。'}

In [87]:
# get the domain2text_ranked
domain2texts = defaultdict(list)
for data in raw_data:
    domain2texts[data["domain"]].append(data["text"])

In [88]:
from itertools import combinations

def group_pairwise_combinations(texts, n):
    """
    将 texts 分组为每 n 个一组，并对每组生成所有两两组合。
    
    参数:
        texts (list): 输入文本列表
        n (int): 每组的元素数量

    返回:
        list: 包含所有两两组合的列表，每组是 [[a,b], [a,c], ...]
    """
    result = []
    for i in range(0, len(texts), n):
        group = texts[i:i+n]
        if len(group) < 2:
            continue  # 跳过不足两个元素的组
        pairs = [list(pair) for pair in combinations(group, 2)]
        result.extend(pairs)
    return result


In [89]:
for domain, texts in domain2texts.items():
    idxes = list(range(len(texts)))
    collection = list(zip(idxes, texts))
    pairwise_data = group_pairwise_combinations(idxes, n=4)
    processed_data_domain_dir = processed_data_dir + f"{domain}/"
    mkdir(processed_data_domain_dir)
    save_tsv(collection, processed_data_domain_dir + collection_file_name)
    save_jsonl(pairwise_data, processed_data_domain_dir + grouped_data_file_name)

>>> make dir: data/processed/agri/
>>> save jsonl to data/processed/agri/grouped_data.jsonl: 1200
>>> make dir: data/processed/med/
>>> save jsonl to data/processed/med/grouped_data.jsonl: 1200
>>> make dir: data/processed/emp/
>>> save jsonl to data/processed/emp/grouped_data.jsonl: 1200
>>> make dir: data/processed/gov/
>>> save jsonl to data/processed/gov/grouped_data.jsonl: 1200
>>> make dir: data/processed/edu/
>>> save jsonl to data/processed/edu/grouped_data.jsonl: 1200
>>> make dir: data/processed/cul/
>>> save jsonl to data/processed/cul/grouped_data.jsonl: 1200
>>> make dir: data/processed/liv/
>>> save jsonl to data/processed/liv/grouped_data.jsonl: 1200
>>> make dir: data/processed/eco/
>>> save jsonl to data/processed/eco/grouped_data.jsonl: 1200
>>> make dir: data/processed/tech/
>>> save jsonl to data/processed/tech/grouped_data.jsonl: 1200
>>> make dir: data/processed/econ/
>>> save jsonl to data/processed/econ/grouped_data.jsonl: 1200


# organize the validation data

In [1]:
# used for validation
# 在processed/validation这个文件夹底下了

# rerank the data

In [55]:
import os
import warnings

In [56]:
from koala.utils.call_llm import call_chatbot

chatbot = call_chatbot(llm_model="qwen-plus")

In [57]:
# 测试
chatbot.chat(messages=[{"role":"user", "content":"how do you do?"}])

"I'm just a computer program, so I don't have feelings, but thanks for asking! How can I assist you today?"

In [58]:
def recover_item(collection, grouped_item):
    recovered_item =  [collection[int(i)] for i in grouped_item]
    return recovered_item
    pass

In [59]:
# recover the data
def recover_data(collection, grouped_data):
    recovered_data = []
    for grouped_item in grouped_data:
        recovered_item = recover_item(collection, grouped_item)
        recovered_data.append(recovered_item)
    return recovered_data

In [60]:
def load_data(processed_data_dir):
    domain2material = {}
    for domain in os.listdir(processed_data_dir):
        processed_data_domain_dir = processed_data_dir + f"{domain}/"
        # print(processed_data_domain_dir)
        
        collection_path = processed_data_domain_dir + collection_file_name
        grouped_data_path = processed_data_domain_dir + grouped_data_file_name
        
        if not os.path.exists(grouped_data_path) or not os.path.exists(collection_path):
            warnings.warn(message=f"The dir not ready in {processed_data_domain_dir}")
            continue
        
        collection = load_tsv(collection_path)
        print(f"loaded collection for domain {domain}", len(collection))
        
        grouped_data = load_jsonl(grouped_data_path)
        print(f"loaded grouped_data for domain {domain}", len(grouped_data))
        # recover the data
        # recovered_data = recover_data(collection, grouped_data)
        print("\n")
        
        domain2material[domain] = {"collection":collection, "grouped_data": grouped_data}
    return domain2material
        # recover the data

# 开始大模型排序

In [54]:
from koala.prompt import *
from functools import partial

In [84]:
from koala.utils.time_limit import RateLimiter
from koala.utils.call_llm import call_chatbot
from retrying import retry
import re
from concurrent.futures import ThreadPoolExecutor, as_completed


In [80]:
class ChatClient:
    def __init__(self, chatbot_name, print_resp, rate_limit_count=1155, rate_limit_period=60):
        self.limiter = RateLimiter(rate_limit_count, rate_limit_period)
        self.chatbot = call_chatbot(chatbot_name)
        self.print_resp = print_resp

    def chat(self, prompt):
        if self.limiter is not None:
            self.limiter.acquire()
        messages = [{"role": "user", "content": prompt}]
        resp = self.chatbot.chat(messages)
        return resp

    @retry(stop_max_attempt_number=5, wait_exponential_multiplier=1000, wait_exponential_max=5000)
    def get_valid_structured_output(self, prompt, parse_func=None):
        """尝试生成并解析判决原因字典，最多重试5次"""
        resp = self.chat(prompt)
        if self.print_resp:
            print("RESP:", resp)
        if not parse_func:
            return resp
        parsed_result = parse_func(resp)
        if parsed_result is None:
            raise ValueError("[Parsed error!]")
        return parsed_result

In [85]:
class RankByLLM(ChatClient):
    def __init__(self, chatbot_name, language, style_name, style_definition, print_resp=False, **kwargs):
        super().__init__(chatbot_name, print_resp, **kwargs)
        # self.chatbot = chatbot
        self.language = language
        self.style_name = style_name
        self.style_definition = style_definition
        self.rank_template, self.extract_template = self.init_template()
    
    def init_template(self):
        if self.language == "cn":
            template = cn_template
            extract_template = f"{self.style_name}更强的文本："
        if self.language == "en":
            template = en_template
            extract_template = f"Text with stronger {style_name}: "
        template_partial = partial(template, self.style_name, self.style_definition)
        return template_partial, extract_template
    
    def parse_func(self, text):
        pattern = re.escape(self.extract_template) + r"\s*\[?([A-Z])\]?"
        match = re.search(pattern, text)
        if match:
            result = match.group(1).strip()
            if result in {"A", "B"}:
                return result
        return None
    
    def rank(self, text1, text2):
        '''
        :param text1: 
        :param text2: 
        :return: it will return "A" or "B" or None 
        '''
        rank_template = self.rank_template(text1, text2)
        try:
            rank_output = self.get_valid_structured_output(rank_template, parse_func=self.parse_func)  # 能正确根据parse_func解析出结果
        except Exception as e:
            print("[Generate rank_output error!]", e)
            rank_output = None
        return rank_output
    
    def _compare_pair(self, text1, text2):
        result1 = self.rank(text1, text2)
        result2 = self.rank(text2, text1)
        if result1 == "A" and result2 == "B":
            return 0
        elif result1 == "B" and result2 == "A":
            return 1
        else:
            return None
        
    
    def rank_items(self, recovered_item_list, max_workers=8):
        '''
        :param recovered_item_list: [[text1, text2], ...]
        :return: [0, 1, ...]  # 0 refers to the unchanged order, index 0 is bigger.
        '''
        rank_output_list = [None] * len(recovered_item_list)
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            future_to_index = {
                executor.submit(self._compare_pair, item[0], item[1]): idx
                for idx, item in enumerate(recovered_item_list)
            }

            for future in as_completed(future_to_index):
                idx = future_to_index[future]
                try:
                    rank_output_list[idx] = future.result()
                except Exception as e:  # 如果某一个线程报错了
                    print(f"Error in future {idx}: {e}")
                    rank_output_list[idx] = None

        return rank_output_list

# rank_output_list = []
# for recovered_item in recovered_item_list:
#     rank_output_1 = rank_machine.rank(recovered_item[0], recovered_item[1])
#     rank_output_2 = rank_machine.rank(recovered_item[1], recovered_item[0])
#     # if (output_1 == "A" and output_2 == "B") or (output_1 == "B" and output_2 == "A"):
#     if rank_output_1 == "A" and rank_output_2 == "B":
#         rank_output_list.append(0)
#     elif rank_output_1 == "B" and rank_output_2 == "A":
#         rank_output_list.append(1)
# return rank_output_list

In [86]:
# chatbot = ChatClient(chatbot_name="qwen-plus", print_resp=True, rate_limit_count=1155, rate_limit_period=60)
rank_machine = RankByLLM(chatbot_name="qwen-plus",
                         print_resp=True,
                         language="cn", 
                         style_name="强制性", 
                         style_definition="语气越强硬、措施或要求越详细具体、涉及到硬性的数字标准，则文本的“强制性”越强。")

# extract_result = rank_machine.rank("你今天一定要把这个东西给我了", "没事，你可以考虑考虑这个应该怎么做，再给我个答案。")

In [87]:
# 接下来，加载没ranked的data

In [88]:
domain2material = load_data(processed_data_dir)

0K loaded collection for domain agri 800
loaded grouped_data for domain agri 1200


0K loaded collection for domain econ 800
loaded grouped_data for domain econ 1200


0K loaded collection for domain gov 800
loaded grouped_data for domain gov 1200


0K loaded collection for domain emp 800
loaded grouped_data for domain emp 1200


0K loaded collection for domain cul 800
loaded grouped_data for domain cul 1200


0K loaded collection for domain liv 800
loaded grouped_data for domain liv 1200


0K loaded collection for domain tech 800
loaded grouped_data for domain tech 1200


0K loaded collection for domain edu 800
loaded grouped_data for domain edu 1200


0K loaded collection for domain eco 800
loaded grouped_data for domain eco 1200


0K loaded collection for domain med 800
loaded grouped_data for domain med 1200




In [89]:
len(domain2material)

10

In [91]:
for domain, material in domain2material.items():
    if domain not in {"tech", "edu"}:  # todo: here!
        continue
    
    print(f"Start to process domain: {domain}...")
    processed_data_domain_dir = processed_data_dir + f"{domain}/"
    collection = material["collection"]
    grouped_data = material["grouped_data"]
    recovered_item_list = []
    for idx, grouped_item in enumerate(grouped_data):
        if idx == 500:  # todo: here!
            break
        recovered_item = recover_item(collection, grouped_item)
        recovered_item_list.append(recovered_item)
    
    rank_output_list = rank_machine.rank_items(recovered_item_list=recovered_item_list, max_workers=8)
    assert len(rank_output_list) == len(recovered_item_list)
    
    ranked_grouped_data = []
    for idx, rank_output in enumerate(rank_output_list):
        assert rank_output in {0, 1, None}
        grouped_item = grouped_data[idx]
        if rank_output == 0:
            ranked_grouped_data.append(grouped_item)
        elif rank_output == 1:
            ranked_grouped_data.append(grouped_item[::-1])
        else:
            continue    
    save_jsonl(ranked_grouped_data, processed_data_domain_dir + ranked_grouped_data_file_name)

Start to process domain: tech...
RESP: ```
强制性更强的文本：B

理由：文本B中涉及了具体的数字目标和明确的要求，例如“全社会研发经费投入增长15%”、“推动落地转化60项以上”、“技术合同成交额增长10%以上”、“全年培育省、市研发机构10家以上”、“新增国家级高新技术企业30家以上”等。这些具体数字和目标体现了强烈的执行力和约束力，语气也更偏向于命令性和任务导向。相比之下，文本A更多地停留在政策方向和原则性的描述上，缺乏具体的量化指标和硬性要求，因此其强制性较弱。
```
RESP: ```
强制性更强的文本：B

理由：文本B中明确提出了具体的措施和硬性数字标准，例如“确保增加值增长10%”、“力争万人发明专利拥有量达2.6件以上”，这些具体的数字目标体现了很强的强制性和可执行性。此外，文本B还详细列出了需要建设的具体平台（如产业技术研究院、院士工作站等）以及需要实施的具体工程（如知识产权质量提升工程），进一步增强了其强制性。相比之下，文本A更多地停留在政策方向和原则性的描述上，缺乏具体的目标和措施，因此其强制性较弱。
```
RESP: ```
强制性更强的文本：B

理由：文本B中提到了具体的数量指标和明确的要求，例如“新增5个市级以上工程技术研究中心和10个企业技术中心”、“力争建成5个以上中试基地和合作示范企业”，这些具体数字目标体现了更强的强制性和执行力。相比之下，文本A虽然也涉及发展目标，但更多是方向性和规划性的描述，如“争取更多的大企业大项目入园”、“争取到2015年把我市初步建设成为海南热带特色现代农业示范区”，缺乏明确的量化标准和硬性要求，因此其强制性较弱。
```
RESP: ```
强制性更强的文本：A

理由：文本A中提到了具体的时间节点和目标，例如“争取到2015年把我市初步建设成为海南热带特色现代农业示范区”，这种明确的时间限制和目标设定增强了文本的强制性。此外，文本A多次提到“强化”、“推进”、“争取更多”等词汇，表明了对措施实施的具体要求和决心，显示出较强的执行力和紧迫感。

相比之下，文本B虽然也提到了一些具体的行动方向（如“加快实施创新驱动发展战略”、“支持企业建设技术创新战略联盟”），但缺乏明确的时间节点或硬性指标，更多是以引导和鼓励为主，语气相对柔和，因此其强制性较弱。
```

# split training and validation data

In [21]:
import os

In [24]:
from koala.utils.utils import save_jsonl

In [30]:
training_dir = "data/processed/training/"
train_ratio=0.8

for root, dirs, files in os.walk(training_dir):
    for file in files:
        if ranked_grouped_data_file_name in file:
            grouped_data_ranked_path = root + "/" + file
            print("start to split:", grouped_data_ranked_path)
            grouped_data = load_jsonl(grouped_data_ranked_path)
            # split it!
            training_num = int(len(grouped_data) * train_ratio)
            validation_num = len(grouped_data) - training_num
            print(training_num, validation_num)
            grouped_data_training = grouped_data[:training_num]
            grouped_data_validation = grouped_data[training_num:]
            train_path = root + "/" + ranked_grouped_train_data_file_name
            valid_path = root + "/" + ranked_grouped_valid_data_file_name
            save_jsonl(grouped_data_training, train_path)
            save_jsonl(grouped_data_validation, valid_path)
            print("\n")

start to split: data/processed/training/tech/grouped_data.ranked.jsonl
338 85
>>> save jsonl to data/processed/training/tech/grouped_data.ranked.train.jsonl: 338
>>> save jsonl to data/processed/training/tech/grouped_data.ranked.valid.jsonl: 85


start to split: data/processed/training/edu/grouped_data.ranked.jsonl
336 84
>>> save jsonl to data/processed/training/edu/grouped_data.ranked.train.jsonl: 336
>>> save jsonl to data/processed/training/edu/grouped_data.ranked.valid.jsonl: 84


